<a href="https://colab.research.google.com/github/s-fellner/BSA_Rolex/blob/main/Code/TFIDF_of_sentences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy download fr
!spacy download fr_core_news_sm

     |████████████████████████████████| 14.7MB 12.5MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp37-none-any.whl size=14727027 sha256=b400d9fc39aba05e5cca9dce6be78f014843b4bdc3aa08457edef03dccfd2aac
  Stored in directory: /tmp/pip-ephem-wheel-cache-c3uox3au/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [ ]:
# Loading every module needed :
import numpy as np
import pandas as pd
import re 
import string

import spacy
sp = spacy.load('fr_core_news_sm')

In [ ]:
df_train = pd.read_csv("https://raw.githubusercontent.com/s-fellner/BSA_Rolex/main/Data/Train_data.csv", sep=',')
df_test = pd.read_csv("https://raw.githubusercontent.com/s-fellner/BSA_Rolex/main/Data/Test_data.csv", sep=',')
print(df_train.head())
print(df_test.head())

X_train = df_train['sentence']
X_test = df_test['sentence']

print(X_train.head())
print(X_test.head())

                      sentence difficulty
0           C'est pour quand ?         A1
1      Je pense que c'est bon.         A1
2               C'est pas mal.         A1
3  Qu'est-ce que vous faites ?         A1
4                 C'est bien !         A1
                                            sentence
0  Il est devenu courant de déplorer que la Franc...
1  Sur les flancs et les derrières de la noce, to...
2  J'aime aussi beaucoup sa voix, une voix un peu...
3  En partenariat avec l'INPES (Institut National...
4  Il se réveilla seulement quand il entendit un ...
0             C'est pour quand ?
1        Je pense que c'est bon.
2                 C'est pas mal.
3    Qu'est-ce que vous faites ?
4                   C'est bien !
Name: sentence, dtype: object
0    Il est devenu courant de déplorer que la Franc...
1    Sur les flancs et les derrières de la noce, to...
2    J'aime aussi beaucoup sa voix, une voix un peu...
3    En partenariat avec l'INPES (Institut National...
4    Il se réve

In [ ]:
# Create a list of stopwords
stop_words = spacy.lang.fr.stop_words.STOP_WORDS

# Create a list of punctuation marks
punctuations = string.punctuation

def spacy_tokenizer(sentence):

  # Create token object, which is used to create documents with linguistic annotations.
  mytokens = sp(sentence)

  # Lemmatize each token and convert each token into lowercase
  mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

  # Remove stop words and punctuation
  mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]

  mytokens = [word for word in mytokens if len(word)>1]

  # Return preprocessed list of tokens
  return mytokens

In [ ]:
spl = X_train.sample(10)
display(spl)
for x in spl:
  print(spacy_tokenizer(x))

3837    Que ces acteurs, telle la marque Max Havelaar,...
501                                    Je m'appelle Jean.
2755    L'auteur souhaite encourager les femmes à s'im...
409     J'étais assez calme, j'avais beaucoup de copai...
4743    Nous voulûmes donner suite à sa requête, mais ...
3394    L'alimentation jouerait donc un rôle dans l'ap...
4747    Et Kant ajoute pour préciser : "(.) si la loi ...
2700    Il a rencontré François 1er pendant une visite...
4171    Le ministre de l'Instruction publique cherchai...
4705    Pour de multiples raisons, l'éducation s'est f...
Name: sentence, dtype: object

['acteur', 'marque', 'max', 'havelaar', 'croire', 'feignent', 'croire', 'disposer', 'rapport', 'force', 'nécessaire', 'infléchir', 'pratique', 'multinationale', 'droit']
['appelle', 'jean']
['auteur', 'souhaite', 'encourager', 'femme', 'impliquer', 'mathématique']
['calme', 'copain', 'passer', 'heure', 'jour']
['vouloir', 'donner', 'suite', 'requête', 'devoir', 'résoudre', 'taire', 'aboutissant']
['alimentation', 'jouer', 'rôle', 'apparition', 'trouble', 'mental']
['kant', 'ajoute', 'préciser', 'loi', 'moral', 'clairement', 'conçue', 'raison', 'croirions', 'jamais', 'autoriser', 'admettre', 'chose', 'liberté']
['rencontrer', 'françois', 'visite', 'calais']
['ministre', 'instruction', 'public', 'chercher', 'soustraire', 'enfant', 'enseignement', 'catholique', 'hérité', 'ii', 'république']
['multiple', 'raison', 'éducation', 'focaliser', 'spécialisation']


In [ ]:
# Tokenize sentences
df_train['tokenized'] = df_train['sentence'].apply(spacy_tokenizer)
df_test['tokenized'] = df_test['sentence'].apply(spacy_tokenizer)
df_train.to_csv('Train_tokenized.csv', encoding='utf-8', index=False)
df_test.to_csv('Test_tokenized.csv', encoding='utf-8', index=False)

In [ ]:
def counterize(df, col):

  wordnb = []
  cognate = []
  punctuation = []

  x=0
  while x < df.shape[0]:
    cogs = []
    wordnb.append(len(re.findall(r'\w+', df[col][x])))
    punctuation.append(len(re.findall(r'[.?\-",]+', df[col][x])))
    for token in df['tokenized'][x]:
      if token in cognates:
        cogs.append(token)
    cognate.append(cogs)
    x+=1

  df['wordnb'] = wordnb
  df['cognate'] = cognate
  df['cognatenb'] = df.cognate.map(len)
  df['cognateratio'] = df['cognatenb'] / df['wordnb']
  df['punctuation'] = punctuation
  df['token_nb'] = df.tokenized.map(len)
  df['tokenratio'] = df['token_nb'] / df['wordnb']

In [ ]:
# Create the TFIDF vector
tfidf_vector = TfidfVectorizer(ngram_range=(1,1), min_df=3, max_df=1.0, analyzer='word', sublinear_tf=True, norm='l2', tokenizer=spacy_tokenizer)

# TFIDF'isation of texts
X_train_vec = pd.DataFrame(tfidf_vector.fit_transform(X_train).todense())
X_test_vec = pd.DataFrame(tfidf_vector.transform(X_test).todense())

# Saving tables
X_train_vec.to_csv('Train_tfidf.csv', encoding='utf-8', index=False)
X_test_vec.to_csv('Test_tfidf.csv', encoding='utf-8', index=False)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3078,3079,3080,3081,3082,3083,3084,3085,3086,3087,3088,3089,3090,3091,3092,3093,3094,3095,3096,3097,3098,3099,3100,3101,3102,3103,3104,3105,3106,3107,3108,3109,3110,3111,3112,3113,3114,3115,3116,3117
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.314221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Saving the vector and exporting it in pickle
import pickle

pickle.dump(tfidf_vector, open("tfidf.pickle", "wb"))

In [ ]:
# Test to load the vector
tfidf_vec = pickle.load(open("tfidf.pickle", "rb"))

In [ ]:
# Trying it
tt = []

tt.append('Ceci est un test !')

tt_vec = pd.DataFrame(tfidf_vec.transform(tt).todense())